### Import Required Libraries and Set Up Environment Variables

In [25]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [26]:
# Set environment variables from the .env in the local environment
load_dotenv()
NYT_API_KEY = os.getenv("NYT_API_KEY")
TMDB_API_KEY = os.getenv("TMDB_API_KEY")

In [27]:
type(NYT_API_KEY)

str

### Access the New York Times API

In [28]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name: "Movies" AND type_of_material: "Review" AND headline: "love"'

# Use a sort filter, sort by newest
sort = "newest"
# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# query_url = (f"{url}api-key={NYT_API_KEY}&q={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}")
query_url = (
    f"{url}api-key={NYT_API_KEY}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [32]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    # page_query = f"&page={page}"
    query_url = f"{query_url}&page={str(page)}"
    # Make a "GET" request and retrieve the JSON
    reviews_list = requests.get(query_url).json()
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        reviews = response.json()
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page + 1}")
    except Exception as e:
        # Print the page number that had no results then break from the loop
        print(f"No results found for page {page + 1}: {e}")
        break
        

{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Diaries (Movie)', 'rank': 2, 'major': 'N'}, {'name': 'persons', 'value': 'Diment, Valentin Javier', 'rank': 3, 'major': 'N'}], 'pub_date': '2023-05-25T11:00:03+0000', 'byline': {'original': 'By Jeannette Catsoulis', 'person': [{'firstname': 'Jeannette', 'middlename': None, 'lastname': 'Catsoulis', 'qua

In [33]:

# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [62]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list = reviews
df_reviews = pd.json_normalize(reviews_list)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'fir

In [65]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include "review" to avoid cutting title early
def extract_title(headline):
    start_index = headline.find('\u2018') + len('\u2018')
    end_index = headline.find('review') + len('review')
    if end_index != -1:
        end_index += len('review')
    else:
        return headline  # Return the original headline if " Review" is not found
    return headline[start_index:end_index]
# Create the new "title" column and apply the extract_title function
df_reviews['title'] = df_reviews["headline.main"].apply(extract_title)
df_reviews[['headline.main', 'title']].head()

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attach
1,Review: ‘What’s Love Got to Do With It?’ Proba...,Wh
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Li
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist’
4,‘Other People’s Children’ Review: True Romance,Other Peop


In [66]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews["keywords"] = df_reviews["keywords"].apply(extract_keywords)
df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attach
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Wh
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Li
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other Peop
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True L
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost W
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,subject: Movies;creative_works: Your Place or ...,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,subject: Documentary Films and Programs;subjec...,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,

In [69]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df_reviews['title'].to_list()
title_list

['The Attach',
 'Wh',
 'You Can Li',
 'A Tourist’',
 'Other Peop',
 'One True L',
 'The Lost W',
 'A Thousand',
 'Your Place',
 'Love in th']

### Access The Movie Database API

In [70]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + TMDB_API_KEY

In [72]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0

# Loop through the titles
for title in title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)

   # Include a try clause to search for the full movie details. 
    # Use the except clause to print out a statement if a movie is not found.
    try:
        # Get movie id
        movie_id = response.json()["results"][0]["id"]

        # Make a request for the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}"
        movie_details_response = requests.get(movie_details_url)

        # Execute "GET" request with url
        movie_details = movie_details_response.json()

        # Extract the genre names into a list
        genres = [genre["name"] for genre in movie_details["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang["english_name"] for lang in movie_details["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country["name"] for country in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_details["title"],
            "overview": movie_details["overview"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })

        # Print out the title that was found
        print(f"Found details for movie: {title}")

    except IndexError:
        print(f"Movie not found: {title}")

# Print the tmdb_movies_list
for movie in tmdb_movies_list:
    print(movie)



Found details for movie: The Attach
Found details for movie: Wh
Found details for movie: You Can Li
Found details for movie: A Tourist’
Found details for movie: Other Peop
Found details for movie: One True L
Found details for movie: The Lost W
Found details for movie: A Thousand
Found details for movie: Your Place
Found details for movie: Love in th
{'title': "The Making of 'Pinocchio': No Strings Attached", 'overview': 'Documentary focusing on the making-of the 1940n adaptation of Pinocchio by the Disney studio, often considered the artistic pinnacle of the Disney feature.', 'genres': ['Documentary'], 'spoken_languages': [], 'production_countries': []}
{'title': 'SPY x FAMILY CODE: White', 'overview': "While under the guise of taking his family on a weekend winter getaway, Loid's attempt to make progress on his current mission Operation Strix proves difficult when Anya mistakenly gets involved and triggers events that threaten world peace.", 'genres': ['Animation', 'Comedy', 'Adventur

In [73]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Making of 'Pinocchio': No Strings Attached",
        "overview": "Documentary focusing on the making-of the 1940n adaptation of Pinocchio by the Disney studio, often considered the artistic pinnacle of the Disney feature.",
        "genres": [
            "Documentary"
        ],
        "spoken_languages": [],
        "production_countries": []
    },
    {
        "title": "SPY x FAMILY CODE: White",
        "overview": "While under the guise of taking his family on a weekend winter getaway, Loid's attempt to make progress on his current mission Operation Strix proves difficult when Anya mistakenly gets involved and triggers events that threaten world peace.",
        "genres": [
            "Animation",
            "Comedy",
            "Adventure",
            "Action"
        ],
        "spoken_languages": [
            "Japanese"
        ],
        "production_countries": [
            "Japan"
        ]
    },
    {
        "title": "You Can Live For

In [74]:
# Convert the results to a DataFrame table
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

# Print the DataFrame
print(df_tmdb_movies)

                                            title  \
0  The Making of 'Pinocchio': No Strings Attached   
1                        SPY x FAMILY CODE: White   
2                            You Can Live Forever   
3                                         Tourist   
4                         Other People's Children   
5                                  One True Loves   
6       Winx Club: The Secret of the Lost Kingdom   
7                     A Thousand Times Good Night   
8                              Your Place or Mine   
9                       Love Is in the Green Wind   

                                            overview  \
0  Documentary focusing on the making-of the 1940...   
1  While under the guise of taking his family on ...   
2  When Jaime, a gay teenager, is sent to live in...   
3  Story of Russian military advisors in the Cent...   
4  Rachel loves her life, her students, her frien...   
5  Emma and Jesse are living the perfect life tog...   
6  Sixteen years ago the

### Merge and Clean the Data for Export

In [75]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, on="title", how="inner")

# Print the merged DataFrame
print(merged_df)

Empty DataFrame
Columns: [web_url, snippet, source, keywords, pub_date, word_count, headline.main, headline.kicker, headline.content_kicker, headline.print_headline, headline.name, headline.seo, headline.sub, byline.original, byline.person, byline.organization, title, overview, genres, spoken_languages, production_countries]
Index: []

[0 rows x 21 columns]


In [76]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']


# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    
    # Loop through characters to remove
    for char in chars_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
print(merged_df)


NameError: name 'chars_to_remove' is not defined

In [77]:
# Drop "byline.person" column
merged_df.drop(columns='byline.person', inplace=True)


In [78]:
# Delete duplicate rows
merged_df.drop_duplicates(inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Print the DataFrame after deleting duplicates and resetting index
print(merged_df)


Empty DataFrame
Columns: [web_url, snippet, source, keywords, pub_date, word_count, headline.main, headline.kicker, headline.content_kicker, headline.print_headline, headline.name, headline.seo, headline.sub, byline.original, byline.organization, title, overview, genres, spoken_languages, production_countries]
Index: []


In [79]:
# Export data to CSV without the index
merged_df.to_csv('merged_movie_data.csv', index=False)